In [ ]:
# @Author:  zihaowang
# @Email:   zihao.wang20@alumni.imperial.ac.uk
# @Website: www.wangzihao.org
# @Date:    2021-01-21 23:43:14
# @Last Modified by:   zihaowang
# @Last Modified time: 2021-03-28 22:24:36

import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
import sys
sys.path.append("../src")

import cv2
import keras
import numpy as np
import matplotlib.pyplot as plt
import segmentation_models as sm
sm.set_framework('keras')
import ad_seg_utils as seg_utils
import bb_eval_utils as eval_utils
import csv
import math
import matplotlib.path as pltPath

Using TensorFlow backend.


Segmentation Models: using `keras` framework.


In [ ]:
SEGMENTATION = "skin"
PERTURBATION = "noise"

In [ ]:
def compute_robustness_iou(true_mask, ref_boxes, perturbed_boxes):
	ref_region = np.zeros(shape=true_mask.shape[0:2], dtype=np.int8)
	perturbed_region = np.zeros(shape=true_mask.shape[0:2], dtype=np.int8)

	# coordinate transformation
	for box in ref_boxes:
		for point in box:
			point[1] = true_mask.shape[0] - point[1]
	for box in perturbed_boxes:
		for point in box:
			point[1] = true_mask.shape[0] - point[1]

	# iterate over all pixels to find which of them are contained in reference prediction
	# and which of them are contained in perturbed prediction
	for i in range(true_mask.shape[0]):
		for j in range(true_mask.shape[1]):
			# iterate through all the boxes to check if pixel is inside any of them
			for box in ref_boxes:
				path = pltPath.Path([box[0], box[1], box[2], box[3]])
				if path.contains_points([[j, true_mask.shape[0] - i]]):
					# count pixels in both ground truth mask and boxes (TP)
					ref_region[i, j] += 1
					break
			for box in perturbed_boxes:
				path = pltPath.Path([box[0], box[1], box[2], box[3]])
				if path.contains_points([[j, true_mask.shape[0] - i]]):
					# count pixels in both ground truth mask and boxes (TP)
					perturbed_region[i, j] += 1
					break

	intersection = np.multiply(ref_region, perturbed_region)
	union = np.add(ref_region, perturbed_region)
# 	print("max: ", np.max(intersection))
	nb_intersection = np.sum([pixel > 0 for pixel in intersection])
	nb_union = np.sum([pixel > 0 for pixel in union])
	IoU = 0 if nb_union == 0 else nb_intersection / nb_union

	print("IoU: ", IoU)
# 	plt.figure(dpi=1200)
# 	plt.tight_layout()
	
	plt.subplot(333)
	plt.box(False)
	plt.axis('off')
	plt.imshow(ref_region)
	plt.title("unpert boxes")
    

	plt.subplot(336)
	plt.box(False)
	plt.axis('off')
	plt.imshow(perturbed_region)
	plt.title(PERTURBATION + " boxes")

	plt.subplot(337)
	plt.box(False)
	plt.axis('off')
	plt.imshow(union)
	plt.title("union")

	plt.subplot(338)
	plt.box(False)
	plt.axis('off')
	plt.imshow(intersection)
	plt.title("intersection")
	
    
	return IoU

In [ ]:

################################ Model loading (this part will be replaced with new data type soon) ################################

print("Program initiating... type of segmentation: " + SEGMENTATION + ", type of perturbation: " + PERTURBATION)

BIN_SEG = True
CLASSES = ['background', 'skin']
WEIGHTS = np.array([1, 1])
target_idx = 1
MODEL_NAME = '/epoch34Save/skin_old_cb2_ce/old_cb2_skin_ce.h5'
BACKBONE = 'efficientnetb3'
LR = 0.0001

# set parameters based on the type of segmentation
if SEGMENTATION == 'SKIN' or SEGMENTATION == 'skin':
    pass
elif SEGMENTATION == 'AD' or SEGMENTATION == 'ad':
    BIN_SEG = False
    target_idx = 2
    CLASSES = ['background', 'skin', 'eczema']
    WEIGHTS = np.array([1, 1, 1])
    MODEL_NAME = '/old_cb2_ad.h5'
else:
    print('Unexpected type of segmentation, should be either skin or ad\n program terminated')

preprocess_input = sm.get_preprocessing(BACKBONE)
"""# Model Evaluation"""
# config PROJ_DIR according to your environment
PROJ_DIR = "/path_to_project_dir"
PRED_DIR = os.path.join(PROJ_DIR, 'output/predictions/' + SEGMENTATION + '_test')
BB_DIR = os.path.join(PROJ_DIR, 'output/bounding_boxes')
EVAL_DIR = os.path.join(PROJ_DIR, 'output/evaluations')
MODEL_DIR = os.path.join(PROJ_DIR, 'output/models')
DATA_DIR = os.path.join(PROJ_DIR, 'data')

# new dataset
x_ref_dir = os.path.join(DATA_DIR, 'test_set/reals')
y_ref_dir = os.path.join(DATA_DIR, 'test_set/labels')
x_perturb_dir = os.path.join(DATA_DIR, 'perturbed_test_sets/adversarial_test_set_' + PERTURBATION)
print('reading ref images from: ' + str(x_ref_dir))
print('reading perturbed images from: ' + str(x_perturb_dir))

reference_dataset = seg_utils.Dataset(
    x_ref_dir,
    y_ref_dir,
    classes=CLASSES,
    augmentation=None,
    preprocessing=seg_utils.get_preprocessing(preprocess_input),
    is_train=False,
    use_full_resolution=False,
    binary_seg=BIN_SEG,
)

perturbed_dataset = seg_utils.Dataset(
    x_perturb_dir,
    y_ref_dir,
    classes=CLASSES, 
    augmentation=None,
    preprocessing=seg_utils.get_preprocessing(preprocess_input),
    is_train=False,
    use_full_resolution=False,
    binary_seg=BIN_SEG,
)

model = seg_utils.load_model(dir=MODEL_DIR + MODEL_NAME, classes=CLASSES, weights=WEIGHTS)
print('Trained model loaded!')


In [ ]:
################################ Mask prediction and evaluation ################################
"""# Saving Masks Predictions"""
# save all predictions
# clear previous predictions
print('Creating directories and clearing previous masks...')
os.system("mkdir -p " + PRED_DIR)
# os.system("rm " + PRED_DIR + "/*.jpg")
# os.system("rm " + PRED_DIR + "/*.JPG")
# os.system("rm " + BB_DIR + "/*.jpg")
# os.system("rm " + BB_DIR + "/*.JPG")
# os.system("rm " + EVAL_DIR + "/robustness_evaluation_" + SEGMENTATION + "_DA_test" + ".csv")


In [ ]:
print('Done! Now saving new prediction masks...')
# create a list to store a series of IoU values
iou = []

with open(EVAL_DIR + "/robustness_evaluation_" + SEGMENTATION + "_DA_test" + ".csv", 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["Reference file name", "Perturbation file name", "IoU"])
# 		for i in range(len(reference_dataset)):
    low_perf_list = [8, 28, 32, 47, 61]
#     low_perf_list = [15, 33, 54, 63, 73, 161]
    for i in range(len(low_perf_list)):
        low_perf_list[i] = low_perf_list[i] - 2
    
    for i in low_perf_list:
        
        # prediction for reference and perturbed images
        ref_image, gt_mask = reference_dataset[i]
        ref_image = np.expand_dims(ref_image, axis=0)
        ref_pred = model.predict(ref_image)

        perturbed_image, _ = perturbed_dataset[i]
        perturbed_image = np.expand_dims(perturbed_image, axis=0)
        perturbed_pred = model.predict(perturbed_image)
    
        # extract the last index to decide which mask to output. [0: background; 1: skin; 2: eczema]
        ref_pred_img = ref_pred[0, :, :, target_idx]
        ref_pred_img = (ref_pred_img * 255).astype(np.uint8)

        perturbed_pred_img = perturbed_pred[0,:,:,target_idx]
        perturbed_pred_img = (perturbed_pred_img * 255).astype(np.uint8)
        
        
        plt.figure(dpi=400)
        left  = 0  # the left side of the subplots of the figure
        right = 0.8    # the right side of the subplots of the figure
        bottom = 0   # the bottom of the subplots of the figure
        top = 1      # the top of the subplots of the figure
        wspace = 0   # the amount of width reserved for blank space between subplots
        hspace = 0.3   # the amount of height reserved for white space between subplots
        alpha = 0.5

        plt.subplots_adjust(left=left, bottom=bottom, right=right, top=top, wspace=wspace, hspace=hspace)
        
        plt.subplot(331)
        plt.title('unpert img')
        plt.box(False)
        plt.axis('off')
        plt.imshow(cv2.cvtColor(cv2.imread(os.path.join(x_ref_dir, reference_dataset.images_ids[i])),cv2.COLOR_BGR2RGB))
        
        plt.subplot(332)
        plt.title('unpert pred')
        plt.box(False)
        plt.axis('off')
        plt.imshow(ref_pred_img)
        
        
        plt.subplot(334)
        plt.title(PERTURBATION + ' img')
        plt.box(False)
        plt.axis('off')
        plt.imshow(cv2.cvtColor(cv2.imread(os.path.join(x_perturb_dir, perturbed_dataset.images_ids[i])),cv2.COLOR_BGR2RGB))
        
        plt.subplot(335)
        plt.title(PERTURBATION + ' pred')
        plt.box(False)
        plt.axis('off')
        plt.imshow(perturbed_pred_img)
        

        # save the images
        cv2.imwrite(os.path.join(PRED_DIR, "pred_" + reference_dataset.images_ids[i]), ref_pred_img)
        cv2.imwrite(os.path.join(PRED_DIR, "pred_" + perturbed_dataset.images_ids[i]), perturbed_pred_img)
        print("saving", reference_dataset.images_ids[i])
        print("saving", perturbed_dataset.images_ids[i])
        
        # generate bounding boxes for each predicted mask
        perturbed_boxes = []
        perturbed_pred_img = cv2.imread(os.path.join(PRED_DIR, "pred_" + perturbed_dataset.images_ids[i]))
        gray = cv2.cvtColor(perturbed_pred_img,cv2.COLOR_BGR2GRAY)
        thresh = cv2.threshold(gray,127,255,cv2.THRESH_BINARY)[1]
        perturbed_contours = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        perturbed_contours = perturbed_contours[0] if len(perturbed_contours) == 2 else perturbed_contours[1]

        ref_boxes = []
        ref_pred_img = cv2.imread(os.path.join(PRED_DIR, "pred_" + reference_dataset.images_ids[i]))
        gray = cv2.cvtColor(ref_pred_img, cv2.COLOR_BGR2GRAY)
        thresh = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY)[1]
        ref_contours = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        ref_contours = ref_contours[0] if len(ref_contours) == 2 else ref_contours[1]

        # extract contours from perturbed predictions
        for cntr in perturbed_contours:
            rect = cv2.minAreaRect(cntr)
            box = cv2.boxPoints(rect)
            box = np.int0(box)
            area = cv2.contourArea(cntr)
            # Abandon boxes with too small area
            if seg_utils.crop_filter(area):
                perturbed_boxes.append(box)

        # extract contours from reference predictions
        for cntr in ref_contours:
            rect = cv2.minAreaRect(cntr)
            box = cv2.boxPoints(rect)
            box = np.int0(box)
            area = cv2.contourArea(cntr)
            # Abandon boxes with too small area
            if seg_utils.crop_filter(area):
                ref_boxes.append(box)

        # append IoU to the list
        iou_per_image = compute_robustness_iou(gt_mask, ref_boxes, perturbed_boxes)
        iou.append(iou_per_image)
        writer.writerow([reference_dataset.images_ids[i], perturbed_dataset.images_ids[i], iou_per_image])
        
        stored_name = reference_dataset.images_ids[i]
        stored_name =  stored_name[0:-4] + '_' + SEGMENTATION + '_' + PERTURBATION + '.eps'
        plt.savefig(stored_name, format='eps', bbox_inches='tight', pad_inches=0.2)
        plt.show()

    # append the mean performance to the end of csv
    writer.writerow(['mean', '', np.mean(iou)])
    writer.writerow(['se', '', np.std(iou) / np.sqrt(len(iou))])

    print('Done!')

In [ ]:
print(gt_mask.shape[0:2])

In [ ]:
!nvidia-smi

In [ ]:
!python ../src/train_batch2.py --seg_type skin --train_dir /path_to_project_dir/data/augmented_training_set_corrected --prefix old_cb2